In [1]:
%load_ext lab_black

In [2]:
import subprocess, logging, sys, glob

FORMAT = "%(asctime)s: %(message)s"
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format=FORMAT)

sys.path.append("/home/murrelab/MiD_HiChIP_Project/data_to_publish/hic_analysis_utils/")

import region_coverage
import pyranges as pr
import pandas as pd
import numpy as np

2022-04-23 23:50:10,665: Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-04-23 23:50:10,667: NumExpr defaulting to 8 threads.


In [3]:
all_bam = {
    bam_path.split("/")[1].split(".trim")[0]: bam_path
    for bam_path in glob.glob("bam/*bam")
}

Merge all simulated B ATAC peaks

In [4]:
merged = None
for sample_label in all_bam.keys():
    macs2_peak_file = f"macs2_results/{sample_label}/{sample_label}_peaks.narrowPeak"
    if "_M" in sample_label:
        print("Not include", sample_label)
        continue

    count = sum(1 for line in open(macs2_peak_file))
    if count < 10_000:
        print("Not include", sample_label, " because of low signal")
        continue

    peaks = (
        pd.read_csv(
            macs2_peak_file,
            header=None,
            sep="\t",
        )
        .iloc[:, [0, 1, 2]]
        .rename(columns={0: "Chromosome", 1: "Start", 2: "End"})
    )
    if merged is None:
        merged = peaks
    else:
        merged = pr.PyRanges(pd.concat([merged, peaks], ignore_index=True)).cluster()
        merged = merged.df.groupby("Cluster").apply(
            lambda df: pd.Series(
                {
                    "Chromosome": df.Chromosome.iloc[0],
                    "Start": df.Start.min(),
                    "End": df.End.max(),
                }
            )
        )

Not include ATAC308_BG_FR105.1_CTLA4_sB  because of low signal
Not include ATAC033_CB_FR008.1_CTLA4_sB  because of low signal
Not include ATAC250_CB_FR083.1_CVID_M
Not include ATAC024_CB_FR005.1_NFKB1_sB  because of low signal
Not include ATAC247_CB_FR082.1_TACI_M


One-hot encoding for peak origins

In [6]:
for sample_label in all_bam.keys():
    macs2_peak_file = f"macs2_results/{sample_label}/{sample_label}_peaks.narrowPeak"
    if "_M" in sample_label:
        continue
    count = sum(1 for line in open(macs2_peak_file))
    if count < 10_000:
        continue

    merged = pr.PyRanges(merged)
    peaks = pr.PyRanges(
        pd.read_csv(
            macs2_peak_file,
            header=None,
            sep="\t",
        )
        .iloc[:, [0, 1, 2]]
        .rename(columns={0: "Chromosome", 1: "Start", 2: "End"})
    )
    merged = merged.coverage(peaks, overlap_col=sample_label, fraction_col="F").df.drop(
        columns="F"
    )
    merged[sample_label] = merged[sample_label].astype(bool)

In [9]:
merged_raw = merged.copy()

Count coverage at peaks

In [15]:
merged = pr.PyRanges(merged_raw.copy())
for sample_label, bam in all_bam.items():
    macs2_peak_file = f"macs2_results/{sample_label}/{sample_label}_peaks.narrowPeak"
    if "_M" in sample_label:
        continue
    count = sum(1 for line in open(macs2_peak_file))
    if count < 10_000:
        continue

    # transform bam to bed format
    with open("temp.bed", "w") as o:
        tobed = subprocess.Popen(
            f"bedtools bamtobed -i {bam}",
            shell=True,
            stdout=subprocess.PIPE,
        )
        subprocess.run(
            "cut -f1-3",
            shell=True,
            stdin=tobed.stdout,
            stdout=o,
        )
        tobed.wait()
    scaled_counts = region_coverage.region_coverage(
        merged, "temp.bed", fill_zeros=False
    )
    merged = merged.insert(pd.Series(data=scaled_counts, name=f"Count_{sample_label}"))

In [18]:
merged.to_csv("merge_peaks.RPM_count.txt", sep="\t")

Raw counts

In [19]:
merged = pr.PyRanges(merged_raw.copy())
for sample_label, bam in all_bam.items():
    macs2_peak_file = f"macs2_results/{sample_label}/{sample_label}_peaks.narrowPeak"
    if "_M" in sample_label:
        continue
    count = sum(1 for line in open(macs2_peak_file))
    if count < 10_000:
        continue

    # transform bam to bed format
    with open("temp.bed", "w") as o:
        tobed = subprocess.Popen(
            f"bedtools bamtobed -i {bam}",
            shell=True,
            stdout=subprocess.PIPE,
        )
        subprocess.run(
            "cut -f1-3",
            shell=True,
            stdin=tobed.stdout,
            stdout=o,
        )
        tobed.wait()
    scaled_counts = region_coverage.region_coverage(
        merged, "temp.bed", fill_zeros=False, scale_to=0
    )
    merged = merged.insert(pd.Series(data=scaled_counts, name=f"Count_{sample_label}"))

In [28]:
for column in merged.columns:
    if "Count" in column:
        merged = merged.assign(column, lambda df: df[column].astype(int))

In [30]:
merged.to_csv("merge_peaks.raw_count.txt", sep="\t")

In [9]:
import pandas as pd
import numpy as np

In [2]:
count_tbl = pd.read_csv("merge_peaks.raw_count.txt", sep="\t")

In [7]:
nt_samples = count_tbl.columns.str.match(r"^ATAC.*ND")
nfkb1_samples = count_tbl.columns.str.match(r"^ATAC.*NFKB1")

In [20]:
count_tbl.loc[
    count_tbl.loc[:, np.logical_or(nt_samples, nfkb1_samples)].apply(np.any, axis=1), :
]

,Chromosome,Start,End,ATAC188_CB_FR062.1_ND_sB,ATAC088_CB_FR028.1_CVID_sB,ATAC111_CB_FR036.1_NFKB1_sB,ATAC212_CB_FR071.1_CVID_sB,ATAC179_CB_FR059.1_TACI_sB,ATAC270_CB_FR090.1_CVID_sB,ATAC128_CB_FR042.1_CVID_sB,...,Count_ATAC027_CB_FR006.1_TACI_sB,Count_ATAC191_CB_FR063.1_ND_sB,Count_ATAC085_CB_FR027.1_CTLA4_sB,Count_ATAC273_CB_FR091.1_HIES_sB,Count_ATAC105_CB_FR034.1_NFKB2_sB,Count_ATAC298_BG_FR101.1_NFKB1_sB,Count_ATAC242_CB_FR080.1_CVID_sB,Count_ATAC185_CB_FR061.1_ND_sB,Count_ATAC224_CB_FR075.1_ND_sB,Count_ATAC218_CB_FR073.1_CVID_sB
0,chr1,10043,10267,True,True,True,False,True,False,False,...,16,15,12,9,5,12,13,11,13,11
1,chr1,180704,180974,True,True,False,False,False,True,False,...,12,11,21,10,5,13,15,14,8,10
2,chr1,181330,181577,True,True,True,True,True,False,False,...,3,15,1,3,9,2,9,12,14,7
6,chr1,629820,630076,True,True,True,True,True,True,True,...,57,56,57,55,54,62,56,51,55,55
7,chr1,630217,630417,False,False,False,False,False,False,False,...,4,1,0,3,0,3,0,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147445,chrY,56734665,56734914,True,False,False,False,False,True,False,...,24,0,0,20,0,0,30,0,0,0
147446,chrY,56742630,56742792,True,False,False,False,False,True,False,...,11,0,0,5,0,0,11,0,0,0
147447,chrY,56763398,56763651,True,False,False,False,False,True,False,...,31,1,0,19,0,0,24,0,0,0
147448,chrY,56837247,56837352,False,False,False,False,False,False,False,...,9,4,6,3,8,7,10,3,3,5
